# Running a Llama2 model locally with API seving via NGrok


In [ ]:
!pip install aiohttp pyngrok

In [6]:
!ngrok config add-authtoken # your NGrok auth token

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
!curl -fsSL https://ollama.com/install.sh | sh

In [ ]:
import os
import asyncio

# Set LD_LIBRARY_PATH so the system NVIDIA library
os.environ.update({'LD_LIBRARY_PATH': '/usr/lib64-nvidia'})

async def run_process(cmd):
  print('>>> starting', *cmd)
  p = await asyncio.subprocess.create_subprocess_exec(
      *cmd,
      stdout=asyncio.subprocess.PIPE,
      stderr=asyncio.subprocess.PIPE,
  )

  async def pipe(lines):
    async for line in lines:
      print(line.strip().decode('utf-8'))

  await asyncio.gather(
      pipe(p.stdout),
      pipe(p.stderr),
  )

await asyncio.gather(
    run_process(['ollama', 'serve']),
    run_process(['ollama', 'pull', 'llama2']),
    run_process(['ngrok', 'http', '--log', 'stderr', '11434']),
)

In [ ]:
# Prompt it
!curl http://NGROK_ENDPOINT:11434/api/generate -d '{ "model": "llama2", "prompt":"tell me a joke" }'
